In [1]:
#before that run this command if pymysql didnt exist in your computer 
#pip install pymysql
import requests
import pandas as pd
import numpy as np
import pymysql

In [2]:
#request api
r = requests.get('https://storage.googleapis.com/zalora-interview-data/bitstampUSD_1-min_data_2012-01-01_to_2020-09-14.csv')

In [3]:
# made CSV file and write it
open('bitcoin.csv', 'wb').write(r.content)

294068701

In [4]:
df = pd.read_csv('F:/python/bitcoin.csv')

In [5]:
headers = ["timestamp","open","high","low","close","vol_btc","vol_currency","weight_price"]
df.columns = headers

In [6]:
#before we remove the value of nan, we have to check the number of empty columns
#df.isnull().sum()

timestamp             0
open            1241716
high            1241716
low             1241716
close           1241716
vol_btc         1241716
vol_currency    1241716
weight_price    1241716
dtype: int64

In [7]:
nan_value = float("Nan")
df.replace("", nan_value, inplace=True)

In [8]:
#because every column have same value, so we just use open column
df.dropna(subset = ["open"], inplace=True)
df.head()

,timestamp,open,high,low,close,vol_btc,vol_currency,weight_price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


In [9]:
#replace timestamp with datetime
df['timestamp']= pd.to_datetime(df['timestamp'],unit='s')
df.replace("",df['timestamp'], inplace=True)

In [10]:
data = pd.DataFrame(df, columns= ['timestamp','open','high','low','close','vol_btc','vol_currency','weight_price'])
#data.head()

,timestamp,open,high,low,close,vol_btc,vol_currency,weight_price
0,2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,2011-12-31 15:50:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,2011-12-31 16:59:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,2011-12-31 17:00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,2012-01-01 04:16:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


In [11]:
#before we connect the database, we must already make a new table after cleaning processing
#connect to database using localhost
database = pymysql.connect(host="localhost",user="root",passwd="", db="bitcoin")
cursor = database.cursor()

In [12]:
#insert data to the table, we must check for each column had the same name between in data frame and table
for row in df.itertuples():
    cursor.execute("INSERT INTO coin (timestamp,open,high,low,close,vol_btc,vol_currency,weight_price) "
  "VALUES (%s,%s,%s,%s,%s,%s,%s,%s)",
                (row.timestamp,
                 row.open,
                 row.high,
                 row.low,
                 row.close,
                 row.vol_btc,
                 row.vol_currency,
                 row.weight_price)    
                )
database.commit()

In [13]:
#after that, we run this query to get the result
cursor.execute("""SELECT 
CASE WHEN a.rnk = b.rnk then a.tanggal end as buying_date,
CASE WHEN a.rnk = b.rnk then a.vol_currency end as buying_price,
CASE WHEN a.rnk = b.rnk then b.tanggal end as selling_date,
CASE WHEN a.rnk = b.rnk then b.vol_currency end as selling_price,
round(((b.vol_currency - a.vol_currency)/b.vol_currency)*100,2) roi
FROM
(SELECT tanggal, open, high, low, close, vol_btc, vol_currency, weight_price, RANK()OVER (ORDER BY tanggal) AS rnk FROM
(SELECT date(timestamp) tanggal, open, high, low, close,vol_btc, vol_currency, weight_price,
RANK()OVER (ORDER BY date(timestamp)) AS rnk
FROM 
(SELECT *,
ROW_NUMBER()OVER (PARTITION BY date(timestamp) ORDER BY date(timestamp) desc) AS rnk  
from coin) a
where rnk =1 order by tanggal) a where rnk %2 <> 0) a
JOIN
(SELECT tanggal, open, high, low, close, vol_btc, vol_currency, weight_price, RANK()OVER (ORDER BY tanggal) AS rnk FROM
(SELECT date(timestamp) tanggal, open, high, low, close,vol_btc, vol_currency, weight_price,
RANK()OVER (ORDER BY date(timestamp)) AS rnk
FROM 
(SELECT *,
ROW_NUMBER()OVER (PARTITION BY date(timestamp) ORDER BY date(timestamp) desc) AS rnk  
from coin) a
where rnk =1 order by tanggal) a where rnk %2 = 0) b
ON a.rnk = b.rnk """)

1589

In [24]:
#the result
myresult = cursor.fetchall()

In [40]:
df = pd.DataFrame(myresult, columns = ['buying_date','buying_price','selling_date','selling_price','roi'])
direktori = 'F:\python\data_assesment.csv'
df.to_csv (direktori, index = False, header=True)